# Q1: Basic File ETL (similar to Adobe structure)

## Dataset with the following
    - tsv format
    - evar columns
    - separate header file
    - separate evar mapping file
    
    

In [3]:
!pip install names

  Using cached names-0.3.0.tar.gz (789 kB)
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803688 sha256=072522ef5548b0d59743a97ecc6b8082f4d6c95c9a1fa70846bac1daeebfe2ee
  Stored in directory: /Users/eugene.kim/Library/Caches/pip/wheels/05/ea/68/92f6b0669e478af9b7c3c524520d03050089e034edcc775c2b
Successfully built names


In [4]:
# Make column names
import urllib.request
import random
import json
import names
import pandas as pd
import datetime

word_url = "https://www.mit.edu/~ecprice/wordlist.10000"
response = urllib.request.urlopen(word_url)
long_txt = response.read().decode()
words = long_txt.splitlines()

In [5]:
def random_date(current):
    return current + datetime.timedelta(days=random.randrange(30), hours=random.randrange(12), minutes=random.randrange(60))

startDate = datetime.datetime(2021, 1, 1,0,0)

def make_data(header_file, data_file, mapping_file, n_rows=1000):
    columns = ["date", "merge1", "merge2", "merge3"]
    mappings = {}
    for i in range(1, 51):
        columns.append(f"evar{i}")
        coin_flip = random.choice([0, 1, 2])
        if coin_flip == 1:
            mappings[f"evar{i}"] = random.choice(words)

    print(f"{len(columns)} columns")
    print(f"{len(list(mappings.items())) + 4} non-null columns")

    with open(f'{mapping_file}', 'w') as fp:
        json.dump(mappings, fp)

    d = {}
    for k, v in mappings.items():
        coin_flip = random.choice([0, 1, 2, 3])
        if coin_flip == 0:
            # str
            str_list = [random.choice(words) for i in range(0, n_rows)]
            random.shuffle(str_list)
            d[k] = str_list
        elif coin_flip == 1:
            # int
            int_list = [i for i in range(0, n_rows)]
            random.shuffle(int_list)
            d[k] = int_list
        elif coin_flip == 2:
            # float
            float_list = [i/3.14 for i in range(0, n_rows)]
            random.shuffle(float_list)
            d[k] = float_list
        elif coin_flip == 3:
            # bool
            d[k] = [bool(random.getrandbits(1)) for i in range(0, n_rows)]

    merge1_d = []
    merge2_d = []
    merge3_d = []
    date_d = []
    for i in range(0, 1000):
        merge1_d.append(i)
        merge2_d.append(names.get_full_name())
        merge3_d.append(hex(int(i*1.61)))
        date_d.append(random_date(startDate))
    d['merge1'] = merge1_d
    d['merge2'] = merge2_d
    d['merge3'] = merge3_d
    d['date'] = date_d

    df = pd.DataFrame(d, columns=columns)
    df.dropna(axis=1).info()

    # Export columns
    print(f"Printing header file to {header_file}")
    pd.DataFrame(columns=columns).to_csv(f'{header_file}', index=False)
    # Export data to tsv without header
    print(f"Printing data file to {data_file}")
    df.to_csv(f'{data_file}', sep="\t", header=False, index=False)
    return df

In [6]:
df = make_data('q1/columns.csv', 'q1/data.tsv', 'q1/mappings.json')
df.head()

54 columns
22 non-null columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1000 non-null   datetime64[ns]
 1   merge1  1000 non-null   int64         
 2   merge2  1000 non-null   object        
 3   merge3  1000 non-null   object        
 4   evar6   1000 non-null   float64       
 5   evar7   1000 non-null   object        
 6   evar8   1000 non-null   int64         
 7   evar11  1000 non-null   object        
 8   evar12  1000 non-null   bool          
 9   evar13  1000 non-null   int64         
 10  evar14  1000 non-null   int64         
 11  evar15  1000 non-null   int64         
 12  evar22  1000 non-null   object        
 13  evar28  1000 non-null   float64       
 14  evar30  1000 non-null   bool          
 15  evar31  1000 non-null   object        
 16  evar33  1000 non-null   object        
 17  evar38  1000 non-null 

,date,merge1,merge2,merge3,evar1,evar2,evar3,evar4,evar5,evar6,...,evar41,evar42,evar43,evar44,evar45,evar46,evar47,evar48,evar49,evar50
0,2021-01-06 01:09:00,0,Tyrone Closson,0x0,NaN,NaN,NaN,NaN,NaN,166.878981,...,NaN,rid,NaN,450,NaN,NaN,True,NaN,235.350318,NaN
1,2021-01-07 08:08:00,1,Violet Todd,0x1,NaN,NaN,NaN,NaN,NaN,83.121019,...,NaN,sufficient,NaN,830,NaN,NaN,True,NaN,34.713376,NaN
2,2021-01-07 05:24:00,2,Jamie Dudley,0x3,NaN,NaN,NaN,NaN,NaN,189.171975,...,NaN,eclipse,NaN,45,NaN,NaN,False,NaN,304.458599,NaN
3,2021-01-27 11:05:00,3,Shawn Oppegard,0x4,NaN,NaN,NaN,NaN,NaN,53.503185,...,NaN,jackson,NaN,96,NaN,NaN,True,NaN,315.286624,NaN
4,2021-01-10 07:03:00,4,Maurice Gonzales,0x6,NaN,NaN,NaN,NaN,NaN,120.382166,...,NaN,quoted,NaN,446,NaN,NaN,True,NaN,299.681529,NaN


# Q2: File ETL with bad data

Dataset with the following

    - tsv file format
    - header csv file missing a comma
    - mismatched datatypes in certain columns
    - mismatched rows (add extra data in certain rows, remove some tabs)


In [7]:
make_data('q2/columns.csv', 'q2/data.tsv', 'q2/mappings.json')

def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

with open('q2/columns.csv') as f:
    data = f.read()

comma_i = findOccurrences(data, ',')
comma_to_delete = random.choice(comma_i)
print(data[comma_to_delete-4:comma_to_delete+4])
new_data = data[:comma_to_delete] + data[comma_to_delete+1:]

# Removing comma
with open('q2/columns.csv', 'w') as f:
    f.write(new_data)

with open('q2/columns.csv') as f:
    data = f.read().replace('\n', '')

print(data)

54 columns
20 non-null columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1000 non-null   datetime64[ns]
 1   merge1  1000 non-null   int64         
 2   merge2  1000 non-null   object        
 3   merge3  1000 non-null   object        
 4   evar1   1000 non-null   bool          
 5   evar4   1000 non-null   int64         
 6   evar11  1000 non-null   bool          
 7   evar13  1000 non-null   float64       
 8   evar14  1000 non-null   float64       
 9   evar18  1000 non-null   int64         
 10  evar20  1000 non-null   float64       
 11  evar21  1000 non-null   bool          
 12  evar23  1000 non-null   int64         
 13  evar24  1000 non-null   object        
 14  evar27  1000 non-null   float64       
 15  evar37  1000 non-null   float64       
 16  evar38  1000 non-null   float64       
 17  evar45  1000 non-null 

In [8]:
with open('q2/data.tsv') as f:
    data = f.readlines()

random.shuffle(data)
remove_tab = data[0]
add_extra_data = data[1]
remove_data = data[2]
remove_newline = data[3]
change_dtype = data[4]
print(len(data))

1000


In [9]:
# remove tab
tab_i = findOccurrences(remove_tab, '\t')
tab_to_delete = random.choice(tab_i)
new_data_removed_tab = remove_tab[:tab_to_delete] + remove_tab[tab_to_delete+1:]
data.append(new_data_removed_tab)

In [10]:
# add extra data
extra_data = add_extra_data.split('\t')
extra_data.insert(-1, "extra data")
new_extra_data = '\t'.join(extra_data)
data.append(new_extra_data)

In [11]:
# remove_data
data_removed = remove_data.split('\t')
element_to_remove = random.choice(data_removed)
data_removed.remove(element_to_remove)
new_remove_data = '\t'.join(data_removed)
data.append(new_remove_data)

In [12]:
# remove new line
newline_removed = remove_newline.split('\t')
del newline_removed[-1]
newline_removed_data = '\t'.join(newline_removed)
data.append(newline_removed_data)

In [13]:
# change dtype
dtype_changed = change_dtype.split('\t')
for i, x in enumerate(dtype_changed):
    if not x:
        dtype_changed[i] = "nonnull"  
        break;
dtype_changed_data = '\t'.join(dtype_changed)
data.append(dtype_changed_data)

In [14]:
write_data = data[5:]
random.shuffle(write_data)
print(len(write_data))


1000


In [15]:
with open('q2/data.tsv', 'w') as f:
    for line in write_data:
        f.write(line)

# Q3: Write Function to load multiple files incrementally

- Multiple datasets
- Output to single dataset.

In [16]:
for i in range(1, 10):
    make_data('q3/columns.csv', f'q3/data-{i}.tsv', 'q3/mappings.json')

54 columns
19 non-null columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1000 non-null   datetime64[ns]
 1   merge1  1000 non-null   int64         
 2   merge2  1000 non-null   object        
 3   merge3  1000 non-null   object        
 4   evar11  1000 non-null   bool          
 5   evar18  1000 non-null   object        
 6   evar22  1000 non-null   float64       
 7   evar26  1000 non-null   float64       
 8   evar27  1000 non-null   object        
 9   evar28  1000 non-null   object        
 10  evar31  1000 non-null   object        
 11  evar34  1000 non-null   object        
 12  evar41  1000 non-null   object        
 13  evar42  1000 non-null   float64       
 14  evar44  1000 non-null   bool          
 15  evar46  1000 non-null   bool          
 16  evar47  1000 non-null   object        
 17  evar49  1000 non-null 